<a href="https://colab.research.google.com/github/LucianaNieto/Cambodia_2023/blob/main/ImageAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing & Importing Libraries

In [ ]:
!pip install keract
import keract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pickle
import numpy as np
import pandas as pd
from PIL import Image
import albumentations as A
from IPython.display import SVG
import matplotlib.pyplot as plt
%matplotlib inline
import os, re, sys, random, shutil, cv2



In [ ]:
def augment(width, height):
    transform = A.Compose([
        A.RandomCrop(width=width, height=height, p=1.0),
        A.HorizontalFlip(p=1.0),
        A.VerticalFlip(p=1.0),
        A.Rotate(limit=[60, 300], p=1.0, interpolation=cv2.INTER_NEAREST),
        A.RandomBrightnessContrast(brightness_limit=[-0.2, 0.3], contrast_limit=0.2, p=1.0),
        A.OneOf([
            A.CLAHE (clip_limit=1.5, tile_grid_size=(8, 8), p=0.5),
            A.GridDistortion(p=0.5),
            A.OpticalDistortion(distort_limit=1, shift_limit=0.5, interpolation=cv2.INTER_NEAREST, p=0.5),
        ], p=1.0),
    ], p=1.0)
    
    return transform

In [ ]:
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 16

    if original_image is None and original_mask is None:
        f, ax = plt.subplots(2, 1, figsize=(10, 10), squeeze=True)
        f.set_tight_layout(h_pad=5, w_pad=5)

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(16, 12), squeeze=True)
        plt.tight_layout(pad=0.2, w_pad=1.0, h_pad=0.01)

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original Image', fontsize=fontsize)

        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original Mask', fontsize=fontsize)

        ax[0, 1].imshow(image)
        ax[0, 1].set_title('Transformed Image', fontsize=fontsize)

        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed Mask', fontsize=fontsize)
        
    plt.savefig('sample_augmented_image.png', facecolor= 'w', transparent= False, bbox_inches= 'tight', dpi= 100)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Saving Augmented Images to Disk
I have already performed data augmentation and saved the images. I am not running it's code in this notebook. It is a very time consuming process, so be patient while the code cell runs!

In [ ]:
!mkdir aug_images_test
!mkdir aug_masks_test

In [ ]:
images_dir = '/content/drive/MyDrive/data_cambodia/data2/test/images/'
masks_dir = '/content/drive/MyDrive/data_cambodia/data2/test/mask/'

In [ ]:
file_names = np.sort(os.listdir(images_dir)) 
file_names = np.char.split(file_names, '.')
filenames = np.array([])
for i in range(len(file_names)):
    filenames = np.append(filenames, file_names[i][0])

In [ ]:
def augment_dataset(count):

    transform_1 = augment(512, 512)
    transform_2 = augment(480, 480)
    transform_3 = augment(512, 512)
    transform_4 = augment(800, 800)
    transform_5 = augment(1024, 1024)
    transform_6 = augment(800, 800)
    transform_7 = augment(1600, 1600)
    transform_8 = augment(1920, 1280)
    
    i = 0
    for i in range(count):
        for file in filenames:
            #tile = file.split('_')[1]
            img = cv2.imread(images_dir+file+'.jpg')
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            mask = cv2.imread(masks_dir+file+'.png')
            mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)
            transformed = transform_1(image=img, mask=mask)
            transformed_image = transformed['image']
            transformed_mask = transformed['mask']

                
            cv2.imwrite('./aug_images_test/aug_{}_'.format(str(i+1))+file+'.jpg',cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB))
            cv2.imwrite('./aug_masks_test/aug_{}_'.format(str(i+1))+file+'.png',cv2.cvtColor(transformed_mask, cv2.COLOR_BGR2RGB))

In [ ]:
augment_dataset(8)

In [ ]:
!zip -r aug_images.zip './aug_images_test/'
!zip -r aug_masks.zip './aug_masks_test/'

  adding: aug_images_test/ (stored 0%)
  adding: aug_images_test/aug_8_IMG_1427.jpg (deflated 0%)
  adding: aug_images_test/aug_3_IMG_1433.jpg (deflated 0%)
  adding: aug_images_test/aug_1_IMG_1380.jpg (deflated 1%)
  adding: aug_images_test/aug_7_IMG_1543.jpg (deflated 1%)
  adding: aug_images_test/aug_8_IMG_1479.jpg (deflated 1%)
  adding: aug_images_test/aug_7_IMG_1438.jpg (deflated 1%)
  adding: aug_images_test/aug_6_IMG_1433.jpg (deflated 1%)
  adding: aug_images_test/aug_7_IMG_1433.jpg (deflated 0%)
  adding: aug_images_test/aug_6_IMG_1557.jpg (deflated 0%)
  adding: aug_images_test/aug_5_IMG_1465.jpg (deflated 0%)
  adding: aug_images_test/aug_8_IMG_1498.jpg (deflated 1%)
  adding: aug_images_test/aug_7_IMG_1530.jpg (deflated 4%)
  adding: aug_images_test/aug_2_IMG_1521.jpg (deflated 14%)
  adding: aug_images_test/aug_3_IMG_1438.jpg (deflated 0%)
  adding: aug_images_test/aug_4_IMG_1427.jpg (deflated 1%)
  adding: aug_images_test/aug_7_IMG_1392.jpg (deflated 0%)
  adding: aug_im

In [ ]:
!rm -rf './aug_images_val/'
!rm -rf './aug_masks_val/'

In [ ]:
train_images = "/content/drive/MyDrive/0_imgSeg_Cambodia/data/augData/aug_images_train"
train_masks = "/content/drive/MyDrive/0_imgSeg_Cambodia/data/augData/aug_masks_train"
val_images = "/content/drive/MyDrive/0_imgSeg_Cambodia/data/augData/aug_images_val"
val_masks = "/content/drive/MyDrive/0_imgSeg_Cambodia/data/augData/aug_masks_val"